# Personal tweets sentiment analysis

## This paper presents user interaction with the developed model

interaction interface:<br>
1: You will need to enter screenname in variable "user_screen_name"<br>
2: Enter the required number of current tweets to view in variable "tweets_count"<br>
3: Run first three jupyter cells(if you didn't get http: link on the output, run second and third cell again)<br>
4: Output of the third cell is a link for visualization in the browser. Open this link, later we will need it.<br>
5: Run fourth cell. In the output of this cell, you will see a dataframe. In the "raw_data" column there are actual tweets of the person you are interested in. in the "target" column there are corresponding tweet sentiment scores<br>
6: After full cell execution. follow the link obtained in step 4<br>
### visualization setup
1: In the left part of the interface, find the drop-down list "Color by" and select "Sentiment"<br>
This will help to distinguish between positively and negatively used words in the context of the speech of person being analyzed<br>
Also when choosing a word of interest, after it, in brackets is indicated an accurate assessment positivity of the word used in the contest of this person’s speech<br>
2: On the right side of the interface, locate the “Search” field. With the help of it you can find the words of interest, and analyze words in the original speech space

In [ ]:
user_screen_name = "realDonaldTrump"
tweets_count = 20

In [ ]:
%run thread.py
thread_start()

In [ ]:
url = ! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"
print(url[0]+"/#projector")

In [ ]:
%run main.py
main_func(user_screen_name, tweets_count)